In [37]:
# -*- coding: utf-8 -*-
# cybos related
import sys
import os
proj_dir = "C:\\Users\\sh\\Documents\\devbox\\github\\auto_invest"
sys.path.append(proj_dir)
from src.utils.database import dbMeta

# math
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta

# visualization with Korean
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_loc = os.path.join(proj_dir, "asset\\human_myeongjo.ttf")
font_name = fm.FontProperties(fname=font_loc).get_name()
font = {
    'family': font_name,
    'weight': 'bold',
}
mpl.rc('font', **font)
mpl.rc('text', color='white')
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [38]:
item_cd = 250
sql = """
SELECT 
    *
FROM krx_timeconclude_today
WHERE `item_cd` = {item_cd}
ORDER BY time ASC
;
""".format(item_cd=item_cd)
engine = dbMeta.get_mysql_engine()
snapshot = pd.read_sql(sql, engine)
snapshot.describe()

C:\Users\sh\Anaconda3\lib\site-packages\pymysql\cursors.py:166: Warning: (1366, "Incorrect string value: '\\xB4\\xEB\\xC7\\xD1\\xB9\\xCE...' for column 'VARIABLE_VALUE' at row 480")
  result = self._query(query)


,item_cd,Debi,Dungrak,amount,buyprice,negoprice,sellprice
count,374.0,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,250.0,69.385027,2.163102,119.467914,14093.716578,14119.385027,14144.919786
std,0.0,41.967642,0.369953,375.334229,43.405129,41.967642,42.708175
min,250.0,0.000000,2.000000,1.000000,14000.000000,14050.000000,14050.000000
25%,250.0,50.000000,2.000000,2.000000,14050.000000,14100.000000,14100.000000
50%,250.0,50.000000,2.000000,7.000000,14100.000000,14100.000000,14150.000000
75%,250.0,100.000000,2.000000,46.000000,14150.000000,14150.000000,14200.000000
max,250.0,150.000000,3.000000,3997.000000,14150.000000,14200.000000,14200.000000


In [38]:
plotter = snapshot
plotter = plotter.set_index('time')
plotter.head(10)

target = 'negoprice'
fig, ax = plt.subplots(figsize=(15, 8))
title = r"%s over time" % target
ax.set_title(title, fontsize=30)
ax.set_xlabel('time')
ax.set_ylabel(target)
ax.tick_params(axis='both', labelsize=15, colors='white')

for xs in [ax.xaxis.label, ax.yaxis.label]:
    xs.set_color('white')
    xs.set_fontsize(15)

"""     PLOT LINES      """
ax.plot(plotter[target], 'b', label=target)

"""     GET TWIN AXES    """
ax2 = ax.twinx()
ax2.plot(plotter['amount'], 'r--', label=target)

"""   ADD LEGEND   """
legend = ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
legend.get_frame().set_facecolor('#000000')

plt.show()

In [38]:
# plt.close("all")